# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import requests
import json
import csv
import time

# URL API MOSO
URL = "https://moso.vn/api"
HEADERS = {"Content-Type": "application/json"}

PAYLOAD_TEMPLATE = {
    "action": "find",
    "modelName": "Transaction",  # Lấy dữ liệu từ bảng bất động sản
    "filter": {"status": "open", "listingStatus": "published"},  # Chỉ lấy tin đang đăng
    "options": {"offset": 0, "limit": 100, "sort": {"_createdAt": -1}, "count": True}
}

tat_ca_du_lieu = []
offset = 0
limit = 100

print(" Bắt đầu thu thập dữ liệu từ MOSO...")
while True:
    payload = json.loads(json.dumps(PAYLOAD_TEMPLATE))
    payload["options"]["offset"] = offset
    payload["options"]["limit"] = limit

    res = requests.post(URL, json=payload, headers=HEADERS)
    data = res.json()

    models = data.get("models", [])
    tong_so = data.get("count", 0)

    if not models:
        print(f" Dừng tại offset {offset} (hết dữ liệu)")
        break

    tat_ca_du_lieu.extend(models)
    print(f" Đã lấy được {len(tat_ca_du_lieu)}/{tong_so} căn hộ...")
    offset += limit
    time.sleep(1)

print(f" Hoàn tất: tổng cộng lấy được {len(tat_ca_du_lieu)} căn hộ.")

# ========== XỬ LÝ DỮ LIỆU VÀ CHUYỂN THÀNH DÒNG CHO FILE CSV ==========
dong_du_lieu = []
for item in tat_ca_du_lieu:
    try:
        dia_chi = item.get("pAddress", {}).get("full") or item.get("newPropertyAddress", {}).get("full", "")
        anh = ""
        if item.get("@transaction_TransactionPropertyImage"):
            anh = item["@transaction_TransactionPropertyImage"][0].get("url", "")

        thong_tin = {
            "Ngày đăng": item.get("_createdAt", ""),
            "Loại hình": item.get("pTypeLvl1", "") or item.get("pTypeLvl2", ""),
            "Địa chỉ": dia_chi,
            "Giá (VND)": item.get("price", ""),
            "Diện tích đất (m2)": item.get("pLandArea", ""),
            "Diện tích sàn (m2)": item.get("pArea", ""),
            "Phòng ngủ": item.get("pNumberOfBedrooms", ""),
            "Phòng tắm": item.get("pNumberOfBathrooms", ""),
            "Tình trạng nội thất": item.get("pFurnitureStatus", ""),
            "Giấy tờ pháp lý": item.get("pCertificateType", ""),
            "Tiêu đề": item.get("title", ""),
            "Mô tả": item.get("description", "").replace("\n", " ").strip(),
        }
        dong_du_lieu.append(thong_tin)
    except Exception as e:
        print(f" Lỗi xử lý 1 mẫu: {e}")

# ========== GHI RA FILE CSV ==========
if dong_du_lieu:
    ten_file = "moso_raw_data.csv"
    with open(ten_file, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=list(dong_du_lieu[0].keys()))
        writer.writeheader()
        writer.writerows(dong_du_lieu)

    print(f" Đã lưu toàn bộ dữ liệu vào file: {ten_file}")
else:
    print(" Không có dữ liệu để ghi ra CSV.")

 Bắt đầu thu thập dữ liệu từ MOSO...
 Đã lấy được 100/2200 căn hộ...
 Đã lấy được 200/2200 căn hộ...
 Đã lấy được 300/2200 căn hộ...
 Đã lấy được 400/2200 căn hộ...
 Đã lấy được 500/2200 căn hộ...
 Đã lấy được 600/2200 căn hộ...
 Đã lấy được 700/2200 căn hộ...
 Đã lấy được 800/2200 căn hộ...
 Đã lấy được 900/2200 căn hộ...
 Đã lấy được 1000/2200 căn hộ...
 Đã lấy được 1100/2200 căn hộ...
 Đã lấy được 1200/2200 căn hộ...
 Đã lấy được 1300/2200 căn hộ...
 Đã lấy được 1400/2200 căn hộ...
 Đã lấy được 1500/2200 căn hộ...
 Đã lấy được 1600/2200 căn hộ...
 Đã lấy được 1700/2200 căn hộ...
 Đã lấy được 1800/2200 căn hộ...
 Đã lấy được 1900/2200 căn hộ...
 Đã lấy được 2000/2200 căn hộ...
 Đã lấy được 2100/2200 căn hộ...
 Đã lấy được 2200/2200 căn hộ...
 Dừng tại offset 2200 (hết dữ liệu)
 Hoàn tất: tổng cộng lấy được 2200 căn hộ.
 Đã lưu toàn bộ dữ liệu vào file: moso_raw_data.csv


In [2]:
# ============================================
# PHÂN TÍCH DỮ LIỆU THÔ MOSO (EDA GỐC) — LƯU FILE ẢNH
# ============================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --------------------------------------------
# ⚙ Cấu hình hiển thị
# --------------------------------------------
plt.style.use("seaborn-v0_8-whitegrid")
sns.set(font_scale=1.1)
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams['axes.unicode_minus'] = False

# --------------------------------------------
#  Tạo thư mục lưu ảnh
# --------------------------------------------
output_dir = "charts_raw"
os.makedirs(output_dir, exist_ok=True)

# --------------------------------------------
# 1 Đọc dữ liệu gốc
# --------------------------------------------
df = pd.read_csv("moso_raw_data.csv")

# Chuyển các cột số sang dạng numeric
cols_num = ["Giá (VND)", "Diện tích đất (m2)", "Diện tích sàn (m2)", "Phòng ngủ", "Phòng tắm"]
for col in cols_num:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Điền giá trị rỗng cho cột dạng danh mục
df["Loại hình"] = df["Loại hình"].fillna("Không rõ")
df["Giấy tờ pháp lý"] = df["Giấy tờ pháp lý"].fillna("Không rõ")
if "Tình trạng nội thất" in df.columns:
    df["Tình trạng nội thất"] = df["Tình trạng nội thất"].fillna("Không rõ")

print(f" Đọc thành công {len(df)} bản ghi dữ liệu thô")

# ============================================
# 2 Phân bố loại hình bất động sản
# ============================================
plt.figure(figsize=(8,5))
sns.countplot(y="Loại hình", data=df, order=df["Loại hình"].value_counts().index, palette="coolwarm")
plt.title("Phân bố loại hình bất động sản (Data thô)")
plt.xlabel("Số lượng tin")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_loaihinh.png", dpi=300)
plt.close()

# ============================================
# 3⃣ Phân bố giấy tờ pháp lý
# ============================================
plt.figure(figsize=(8,5))
sns.countplot(y="Giấy tờ pháp lý", data=df, order=df["Giấy tờ pháp lý"].value_counts().index, palette="mako")
plt.title("Phân bố giấy tờ pháp lý (Data thô)")
plt.xlabel("Số lượng tin")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_giayto.png", dpi=300)
plt.close()

# ============================================
# 4️ Biểu đồ tròn: Tình trạng nội thất
# ============================================
if "Tình trạng nội thất" in df.columns:
    interior_counts = df["Tình trạng nội thất"].fillna("Không rõ").value_counts()
    plt.figure(figsize=(7,7))
    colors = sns.color_palette("pastel")[0:len(interior_counts)]
    plt.pie(
        interior_counts.values,
        labels=interior_counts.index,
        autopct=lambda p: f'{p:.1f}%\n({int(p * sum(interior_counts.values) / 100)})',
        startangle=90,
        counterclock=False,
        colors=colors,
        textprops={'fontsize': 12}
    )
    plt.title("Tỷ lệ tình trạng nội thất (Data thô)")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/bieudo_noithat_pie.png", dpi=300)
    plt.close()

# ============================================
# 5️ Phân bố số phòng ngủ
# ============================================
plt.figure(figsize=(8,5))
sns.countplot(x="Phòng ngủ", data=df, palette="crest")
plt.title("Phân bố số phòng ngủ (Data thô)")
plt.xlabel("Số phòng ngủ")
plt.ylabel("Số lượng tin")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_phongngu.png", dpi=300)
plt.close()

# ============================================
# 6️ Phân bố số phòng tắm
# ============================================
plt.figure(figsize=(8,5))
sns.countplot(x="Phòng tắm", data=df, palette="flare")
plt.title("Phân bố số phòng tắm (Data thô)")
plt.xlabel("Số phòng tắm")
plt.ylabel("Số lượng tin")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_phongtam.png", dpi=300)
plt.close()

# ============================================
# 7️ Phân bố giá bất động sản
# ============================================
plt.figure(figsize=(8,5))
sns.histplot(df["Giá (VND)"].dropna(), bins=50, kde=True, color="skyblue")
plt.title("Phân bố giá bất động sản (Data thô)")
plt.xlabel("Giá (VND)")
plt.ylabel("Số lượng tin")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_gia.png", dpi=300)
plt.close()

# ============================================
# 8️ Phân bố diện tích sàn
# ============================================
plt.figure(figsize=(8,5))
sns.histplot(df["Diện tích sàn (m2)"].dropna(), bins=50, kde=True, color="salmon")
plt.title("Phân bố diện tích sàn (Data thô)")
plt.xlabel("Diện tích (m2)")
plt.ylabel("Số lượng tin")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_dientich.png", dpi=300)
plt.close()

# ============================================
# 9️ Boxplot: Giá theo Giấy tờ pháp lý
# ============================================
plt.figure(figsize=(9,5))
sns.boxplot(x="Giấy tờ pháp lý", y="Giá (VND)", data=df, palette="vlag", showfliers=False)
plt.title("So sánh giá theo loại Giấy tờ pháp lý (Data thô)")
plt.xlabel("Giấy tờ pháp lý")
plt.ylabel("Giá (VND)")
plt.xticks(rotation=20)
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_boxplot_giayto.png", dpi=300)
plt.close()

# ============================================
#  Scatterplot: Quan hệ giữa diện tích & giá
# ============================================
plt.figure(figsize=(8,5))
sns.scatterplot(x="Diện tích sàn (m2)", y="Giá (VND)", hue="Loại hình", data=df, alpha=0.6)
plt.title("Quan hệ giữa diện tích và giá (theo loại hình, Data thô)")
plt.xlabel("Diện tích sàn (m²)")
plt.ylabel("Giá (VND)")
plt.legend(title="Loại hình", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_dientich_gia.png", dpi=300)
plt.close()

# ============================================
# 1⃣ Trung bình giá theo loại hình
# ============================================
plt.figure(figsize=(8,5))
avg_price_type = df.groupby("Loại hình")["Giá (VND)"].mean().sort_values(ascending=False)
sns.barplot(x=avg_price_type.values, y=avg_price_type.index, palette="coolwarm")
plt.title("Giá trung bình theo loại hình (Data thô)")
plt.xlabel("Giá trung bình (VND)")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_giatrungbinh_loaihinh.png", dpi=300)
plt.close()

# ============================================
# 1⃣ Heatmap: Ma trận tương quan
# ============================================
num_cols = ["Giá (VND)", "Diện tích đất (m2)", "Diện tích sàn (m2)", "Phòng ngủ", "Phòng tắm"]
corr = df[num_cols].corr()
plt.figure(figsize=(7,5))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Ma trận tương quan giữa các biến số (Data thô)")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_heatmap.png", dpi=300)
plt.close()

# ============================================
# 1 Scatter log-log: Diện tích vs Giá
# ============================================
plt.figure(figsize=(8,5))
sns.scatterplot(x="Diện tích sàn (m2)", y="Giá (VND)", data=df, alpha=0.6, color="teal")
plt.xscale("log")
plt.yscale("log")
plt.title("Quan hệ log-log giữa Diện tích và Giá (Data thô)")
plt.xlabel("Log(Diện tích sàn m²)")
plt.ylabel("Log(Giá VND)")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_loglog_dientich_gia.png", dpi=300)
plt.close()

print(f"✅ Hoàn tất! Tất cả biểu đồ đã lưu tại thư mục '{output_dir}/' 🎨")


 Đọc thành công 2200 bản ghi dữ liệu thô


C:\Users\nam\AppData\Local\Temp\ipykernel_9088\4123279760.py:46: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y="Loại hình", data=df, order=df["Loại hình"].value_counts().index, palette="coolwarm")
C:\Users\nam\AppData\Local\Temp\ipykernel_9088\4123279760.py:57: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y="Giấy tờ pháp lý", data=df, order=df["Giấy tờ pháp lý"].value_counts().index, palette="mako")
C:\Users\nam\AppData\Local\Temp\ipykernel_9088\4123279760.py:89: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x="Phòng ngủ", data=df, 

✅ Hoàn tất! Tất cả biểu đồ đã lưu tại thư mục 'charts_raw/' 🎨


In [7]:
!pip install scikit-learn


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------------------- ------ 51.2/60.8 kB 650.2 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 640.1 kB/s eta 0:00:00
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
    --------------------------------------- 0.1/8.7 MB 944.1 kB/s eta 0:00:10
    --------------------------------------- 0.1/8.7 MB 944.1 kB/s eta 0:00:10
    --------------------------------------- 0.2/8.7 MB 841.6 kB/s eta 0:00:11
   - -------------------------------------- 0.3/8.7 MB 923.9 kB/s eta 0:00:10
   - -------------------------------------- 

In [8]:
import pandas as pd
import re
from sklearn.preprocessing import MinMaxScaler

# === 1. Đọc dữ liệu gốc ===
input_file = "moso_raw_data.csv"
df = pd.read_csv(input_file)

removed_rows = []  # lưu các bản ghi bị loại và lý do

# === 2. Xử lý ngày đăng ===
if "Ngày đăng" in df.columns:
    df["Ngày đăng"] = pd.to_datetime(df["Ngày đăng"], errors="coerce", utc=True)
    df["Ngày đăng"] = df["Ngày đăng"].dt.tz_convert("Asia/Ho_Chi_Minh")
    df["Ngày đăng"] = df["Ngày đăng"].dt.strftime("%d/%m/%Y")

# === 3. Các hàm tiện ích ===
def contains(text, keywords):
    text = str(text).lower()
    return any(kw in text for kw in keywords)

def extract_number(text, pattern):
    match = re.search(pattern, text)
    if match:
        return float(match.group(1).replace(",", "."))
    return None

def fill_interior(row):
    """Điền tình trạng nội thất chuẩn hóa"""
    text = f"{row['Tiêu đề']} {row['Mô tả']}".lower()
    current = str(row["Tình trạng nội thất"]).strip().lower()

    if current in ["furnished", "partially_furnished", "unfurnished"]:
        return current

    if current in ["other", "null", "", "none", "nan", "0"]:
        if contains(text, [
            "đầy đủ nội thất", "full nội thất", "full furnished",
            "đã có nội thất", "có sẵn nội thất", "nội thất cao cấp",
            "có nội thất sẵn"
        ]):
            return "furnished"
        elif contains(text, [
            "nội thất cơ bản", "tặng nội thất", "có 1 phần nội thất",
            "partial", "cơ bản", "một phần nội thất"
        ]):
            return "partially_furnished"
        else:
            return "unfurnished"

    if "đầy đủ" in current or "full" in current:
        return "furnished"
    elif "cơ bản" in current or "partial" in current or "1 phần" in current:
        return "partially_furnished"
    else:
        return "unfurnished"

def extract_room_count(text, room_type):
    text = str(text).lower()
    pattern = rf"(\d+)\s*(phòng\s*{room_type}|{room_type})"
    match = re.search(pattern, text)
    if match:
        return int(match.group(1))
    return None

def extract_area(text):
    text = str(text).lower()
    pattern = r"(\d+(?:[\.,]\d+)?)\s*(m2|m²)"
    match = re.search(pattern, text)
    if match:
        return float(match.group(1).replace(",", "."))
    return None


# === 4. Lọc tin bán đất thật ===
def is_land_listing(row):
    text = f"{row['Tiêu đề']} {row['Mô tả']}".lower()
    if (
        any(kw in text for kw in ["bán đất", "đất nền", "bán đất thổ cư", "lô đất", "miếng đất", "thửa đất"])
        and not any(kw in text for kw in ["căn hộ", "chung cư", "nhà riêng", "nhà phố", "biệt thự", "phòng trọ"])
    ):
        return True
    return False

land_mask = df.apply(is_land_listing, axis=1)
if land_mask.any():
    removed = df[land_mask].copy()
    removed["Lý do loại bỏ"] = "Tin rao bán đất thật"
    removed_rows.append(removed)
    df = df[~land_mask]
    print(f"🧹 Loại bỏ {len(removed)} tin rao bán đất thật")


# === 5. Điền thông tin còn thiếu ===
for i, row in df.iterrows():
    text = f"{row['Tiêu đề']} {row['Mô tả']}"

    # Phòng ngủ
    if pd.isna(row["Phòng ngủ"]) or str(row["Phòng ngủ"]).lower() in ["", "nan", "null", "other", "0"]:
        val = extract_room_count(text, "ngủ")
        df.at[i, "Phòng ngủ"] = val

    # Phòng tắm
    if pd.isna(row["Phòng tắm"]) or str(row["Phòng tắm"]).lower() in ["", "nan", "null", "other", "0"]:
        val = extract_room_count(text, "vệ sinh")
        if val is None:
            val = extract_room_count(text, "tắm")
        df.at[i, "Phòng tắm"] = val

    # Nội thất
    df.at[i, "Tình trạng nội thất"] = fill_interior(row)

    # Diện tích
    if (
        (pd.isna(row["Diện tích đất (m2)"]) or row["Diện tích đất (m2)"] == 0)
        and (pd.isna(row["Diện tích sàn (m2)"]) or row["Diện tích sàn (m2)"] == 0)
    ):
        val = extract_area(text)
        if val and val > 0:
            df.at[i, "Diện tích đất (m2)"] = val


# === 6. Hợp nhất diện tích về 1 cột “Diện tích” ===
df["Diện tích đất (m2)"] = pd.to_numeric(df["Diện tích đất (m2)"], errors="coerce")
df["Diện tích sàn (m2)"] = pd.to_numeric(df["Diện tích sàn (m2)"], errors="coerce")
df["Diện tích"] = df["Diện tích đất (m2)"].fillna(0)
df.loc[df["Diện tích"] == 0, "Diện tích"] = df["Diện tích sàn (m2)"]
df = df[df["Diện tích"] > 0]


# === 7. Loại bỏ giá trị không hợp lệ ===
invalid_mask = (
    (df["Giá (VND)"].isin([0, "0", "null", "nan", "other"])) |
    (df["Diện tích"].isin([0, "0", "null", "nan", "other"])) |
    (df["Tình trạng nội thất"].isin(["other", "null", "nan", "0"]))
)
if invalid_mask.any():
    removed = df[invalid_mask].copy()
    removed["Lý do loại bỏ"] = "Giá trị 0/null/other ở cột quan trọng"
    removed_rows.append(removed)
    df = df[~invalid_mask]
    print(f"🧹 Loại bỏ {len(removed)} dòng có giá trị 0/null/other")


# === 8. Loại bỏ thiếu nặng ===
cols_to_check = ["Giá (VND)", "Diện tích", "Phòng ngủ", "Phòng tắm", "Tình trạng nội thất"]
missing_count = df[cols_to_check].isna().sum(axis=1)
remove_heavy_missing = missing_count >= 3
if remove_heavy_missing.any():
    removed = df[remove_heavy_missing].copy()
    removed["Lý do loại bỏ"] = "Thiếu quá nhiều thông tin quan trọng"
    removed_rows.append(removed)
    df = df[~remove_heavy_missing]
    print(f"🧹 Loại bỏ {len(removed)} bản ghi thiếu dữ liệu nặng")


# === 9. Phát hiện & loại bỏ outlier (IQR) ===
def remove_outliers_iqr_grouped(data, column, label, group_col="Loại hình"):
    results = []
    for name, group in data.groupby(group_col):
        Q1 = group[column].quantile(0.25)
        Q3 = group[column].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        mask_outlier = (group[column] < lower) | (group[column] > upper)
        if mask_outlier.any():
            removed = group[mask_outlier].copy()
            removed["Lý do loại bỏ"] = f"Outlier theo {label} ({name})"
            removed_rows.append(removed)
            print(f"⚠️ Loại {len(removed)} outlier ({label}) trong nhóm {name}")
        results.append(group[~mask_outlier])
    return pd.concat(results, ignore_index=True)

df = remove_outliers_iqr_grouped(df, "Giá (VND)", "Giá")
df = remove_outliers_iqr_grouped(df, "Diện tích", "Diện tích")


# === 10. Giữ lại giá trị gốc trước khi scale ===
df["Giá gốc (VND)"] = df["Giá (VND)"]
df["Diện tích gốc (m2)"] = df["Diện tích"]

# === 11. Chuẩn hoá (MinMaxScaler) ===
scaler = MinMaxScaler()
df[["Giá (VND)", "Diện tích"]] = scaler.fit_transform(df[["Giá (VND)", "Diện tích"]])
print("📏 Đã scale dữ liệu về khoảng [0,1] — nhưng vẫn giữ lại cột giá trị gốc để vẽ biểu đồ.")

# === 12. Giữ cột cần thiết ===
cols_keep = [
    "Ngày đăng", "Loại hình", "Giá (VND)", "Giá gốc (VND)",
    "Diện tích", "Diện tích gốc (m2)",
    "Giấy tờ pháp lý", "Phòng ngủ", "Phòng tắm", "Tình trạng nội thất"
]
df = df[[c for c in cols_keep if c in df.columns]]

# === 13. Lưu kết quả ===
df.to_csv("moso_cleaned.csv", index=False, encoding="utf-8-sig")
print(f"\n✅ Đã lưu file kết quả sạch: moso_cleaned.csv ({len(df)} bản ghi)")

if removed_rows:
    removed_all = pd.concat(removed_rows, ignore_index=True)
    removed_all.to_csv("removed_records.csv", index=False, encoding="utf-8-sig")
    print(f"🧾 Đã lưu file các bản ghi bị loại: removed_records.csv ({len(removed_all)} bản ghi)")
else:
    print("✅ Không có bản ghi nào bị loại.")


🧹 Loại bỏ 119 tin rao bán đất thật
⚠️ Loại 123 outlier (Giá) trong nhóm alley_house
⚠️ Loại 1 outlier (Giá) trong nhóm other
⚠️ Loại 5 outlier (Giá) trong nhóm regular_apartment
⚠️ Loại 42 outlier (Giá) trong nhóm street_front_house
⚠️ Loại 1 outlier (Giá) trong nhóm terraced_house
⚠️ Loại 1 outlier (Giá) trong nhóm villa
⚠️ Loại 59 outlier (Diện tích) trong nhóm alley_house
⚠️ Loại 1 outlier (Diện tích) trong nhóm other
⚠️ Loại 2 outlier (Diện tích) trong nhóm regular_apartment
⚠️ Loại 1 outlier (Diện tích) trong nhóm shophouse
⚠️ Loại 27 outlier (Diện tích) trong nhóm street_front_house
⚠️ Loại 2 outlier (Diện tích) trong nhóm villa
📏 Đã scale dữ liệu về khoảng [0,1] — nhưng vẫn giữ lại cột giá trị gốc để vẽ biểu đồ.

✅ Đã lưu file kết quả sạch: moso_cleaned.csv (1803 bản ghi)
🧾 Đã lưu file các bản ghi bị loại: removed_records.csv (384 bản ghi)


In [11]:
# ============================================
# 📊 TRỰC QUAN HOÁ DỮ LIỆU BĐS SAU KHI LÀM SẠCH — LƯU FILE ẢNH
# ============================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from scipy.stats import pearsonr
import os

# Cấu hình hiển thị
plt.style.use("seaborn-v0_8-whitegrid")
sns.set(font_scale=1.2)
plt.rcParams["figure.figsize"] = (9, 5)
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False

# ============================================
# 1 Đọc dữ liệu và tạo thư mục lưu ảnh
# ============================================
df = pd.read_csv("moso_cleaned.csv")
removed_file_exists = os.path.exists("removed_records.csv")
removed_df = pd.read_csv("removed_records.csv") if removed_file_exists else None

output_dir = "charts_clean"
os.makedirs(output_dir, exist_ok=True)

print(f"✅ Đọc thành công {len(df)} bản ghi sau làm sạch")

# ============================================
# ⃣ Biểu đồ phân bố loại hình & giấy tờ
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.countplot(y="Loại hình", data=df, order=df["Loại hình"].value_counts().index, palette="coolwarm", ax=axes[0])
axes[0].set_title("Phân bố loại hình bất động sản (Sau làm sạch)")
axes[0].set_xlabel("Số lượng tin")

if "Giấy tờ pháp lý" in df.columns:
    sns.countplot(y="Giấy tờ pháp lý", data=df, order=df["Giấy tờ pháp lý"].value_counts().index, palette="crest", ax=axes[1])
    axes[1].set_title("Phân bố giấy tờ pháp lý (Sau làm sạch)")
    axes[1].set_xlabel("Số lượng tin")

plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_loaihinh_giayto.png", dpi=300)
plt.close()

# ============================================
#  Phân bố phòng ngủ & phòng tắm
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(df["Phòng ngủ"].dropna(), bins=20, color="seagreen", ax=axes[0], kde=True)
axes[0].set_title("Phân bố số phòng ngủ (Sau làm sạch)")
sns.histplot(df["Phòng tắm"].dropna(), bins=20, color="coral", ax=axes[1], kde=True)
axes[1].set_title("Phân bố số phòng tắm (Sau làm sạch)")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_phongngu_phongtam.png", dpi=300)
plt.close()

# ============================================
# ⃣ Boxplot giá & diện tích theo loại hình
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.boxplot(x="Loại hình", y="Giá gốc (VND)", data=df, palette="rocket", showfliers=False, ax=axes[0])
axes[0].set_title("Phân bố giá gốc (VND) theo loại hình (Sau làm sạch)")
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=30)
axes[0].yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f"{x/1e9:.1f}"))

sns.boxplot(x="Loại hình", y="Diện tích gốc (m2)", data=df, palette="viridis", showfliers=False, ax=axes[1])
axes[1].set_title("Phân bố diện tích (m²) theo loại hình (Sau làm sạch)")
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=30)

plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_boxplot_gia_dientich.png", dpi=300)
plt.close()

# ============================================
#  Phân bố dữ liệu bị loại (Outlier & lý do)
# ============================================
if removed_df is not None and "Lý do loại bỏ" in removed_df.columns:
    plt.figure(figsize=(9, 5))
    sns.countplot(y="Lý do loại bỏ", data=removed_df, order=removed_df["Lý do loại bỏ"].value_counts().index, palette="magma")
    plt.title("Thống kê lý do loại bỏ dữ liệu (Sau làm sạch)")
    plt.xlabel("Số lượng bản ghi")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/bieudo_lydo_loaidulieu.png", dpi=300)
    plt.close()

# ============================================
# 6 Biểu đồ tròn (Pie Chart) - Tình trạng nội thất
# ============================================
if "Tình trạng nội thất" in df.columns:
    interior_counts = df["Tình trạng nội thất"].value_counts(dropna=False)
    labels = interior_counts.index
    sizes = interior_counts.values

    plt.figure(figsize=(7, 7))
    colors = sns.color_palette("pastel")[0:len(labels)]
    plt.pie(
        sizes,
        labels=labels,
        autopct=lambda p: f'{p:.1f}%\n({int(p * sum(sizes) / 100)})',
        colors=colors,
        startangle=90,
        counterclock=False,
        textprops={'fontsize': 12}
    )
    plt.title("Tỷ lệ tình trạng nội thất (Sau làm sạch)", fontsize=14, fontweight="bold")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/bieudo_noithat_pie.png", dpi=300)
    plt.close()

# ============================================
# ⃣ Thống kê giá trị thiếu còn lại
# ============================================
cols_fill = ["Phòng ngủ", "Phòng tắm", "Tình trạng nội thất", "Diện tích"]
filled_counts = {}
for col in cols_fill:
    if col in df.columns:
        filled_counts[col] = df[col].isna().sum()

plt.figure(figsize=(8, 5))
if sum(filled_counts.values()) > 0:
    sns.barplot(x=list(filled_counts.keys()), y=list(filled_counts.values()), palette="cool")
    plt.title("Số lượng giá trị còn thiếu (Sau làm sạch)")
else:
    plt.text(0.5, 0.5, "✅ Không còn giá trị thiếu sau khi làm sạch dữ liệu",
             ha="center", va="center", fontsize=12, color="green", fontweight="bold")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_missing_values.png", dpi=300)
plt.close()

# ============================================
# 8 Mối quan hệ giữa Diện tích & Giá
# ============================================
r1, p1 = pearsonr(df["Diện tích gốc (m2)"], df["Giá gốc (VND)"])
r2, p2 = pearsonr(df["Diện tích"], df["Giá (VND)"])

fig, axes = plt.subplots(1, 2, figsize=(15, 6))
sns.regplot(x="Diện tích gốc (m2)", y="Giá gốc (VND)", data=df, scatter_kws={"alpha":0.5, "s":40}, line_kws={"color":"red", "lw":2, "ls":"--"}, ax=axes[0])
axes[0].set_title(f"Quan hệ giữa Diện tích và Giá gốc (r={r1:.2f})")
axes[0].yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f"{x/1e9:.1f}"))

sns.regplot(x="Diện tích", y="Giá (VND)", data=df, scatter_kws={"alpha":0.5, "s":40}, line_kws={"color":"blue", "lw":2, "ls":"--"}, ax=axes[1])
axes[1].set_title(f"Quan hệ giữa Diện tích và Giá (chuẩn hoá, r={r2:.2f})")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_dientich_gia.png", dpi=300)
plt.close()

# ============================================
# ⃣ Phân bố giá & diện tích
# ============================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sns.histplot(df["Giá gốc (VND)"]/1e9, bins=40, kde=True, color="orange", ax=axes[0])
axes[0].set_title("Phân bố giá gốc (tỷ VNĐ, Sau làm sạch)")
sns.histplot(df["Diện tích gốc (m2)"], bins=40, kde=True, color="green", ax=axes[1])
axes[1].set_title("Phân bố diện tích (m², Sau làm sạch)")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_gia_dientich.png", dpi=300)
plt.close()

# ============================================
# Heatmap tương quan
# ============================================
corr = df[["Giá gốc (VND)", "Diện tích gốc (m2)", "Phòng ngủ", "Phòng tắm"]].corr(numeric_only=True)
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, cmap="YlGnBu", fmt=".2f")
plt.title("Ma trận tương quan giữa các biến số (Sau làm sạch)")
plt.tight_layout()
plt.savefig(f"{output_dir}/bieudo_heatmap.png", dpi=300)
plt.close()

print(f"\n✅ Hoàn tất! Tất cả biểu đồ đã được lưu vào thư mục '{output_dir}/' 🎨")


✅ Đọc thành công 1803 bản ghi sau làm sạch


C:\Users\nam\AppData\Local\Temp\ipykernel_9088\35057584.py:29: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y="Loại hình", data=df, order=df["Loại hình"].value_counts().index, palette="coolwarm", ax=axes[0])
C:\Users\nam\AppData\Local\Temp\ipykernel_9088\35057584.py:34: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(y="Giấy tờ pháp lý", data=df, order=df["Giấy tờ pháp lý"].value_counts().index, palette="crest", ax=axes[1])
C:\Users\nam\AppData\Local\Temp\ipykernel_9088\35057584.py:58: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x="Loại


✅ Hoàn tất! Tất cả biểu đồ đã được lưu vào thư mục 'charts_clean/' 🎨
